<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
💰 תהליך הסבת שכבת גושי שומה לבנק"ל מודרני 💰 

<h1 style="text-align:center; font-size:150%">
תהליך זה הינו חלק 1 מתוך 2 אשר קורא את נתוני המקור, ממיר את המידע לפי אפיון נדרש ושומר את התוצאה בפורמט טבלאי 

In [ ]:
print('2. Tax Assessment Blocks Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  2.a Reading inputs')

In [ ]:
RawData_folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

<h1 style="text-align:right;font-size:100%">
רשימת גושי שומה

In [ ]:
Tax_List = pd.read_csv(RawData_folder + 'Local\GushShumaList.csv')

<h1 style="text-align:right;font-size:100%">
טבלת גושי שומה

In [ ]:
table_cols = ['GUSH_SHUMA_ID', 'GUSH_NUM', 'GUSH_SUFFIX']

Tax_Table = pd.read_csv(RawData_folder + 'Oracle\gush_shuma.csv',
                        usecols = table_cols,
                        encoding='cp1255')

<h1 style="text-align:right;font-size:100%">
שכבת גושי שומה

In [ ]:
layer_cols = ['GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

Tax_Layer = gpd.read_file(RawData_folder + 'CADSDE\SUB_GUSH_ALL_SHUMA.gdb',
                          layer = 'SUB_GUSH_ALL_SHUMA',
                          include_fields = layer_cols)

Tax_Layer.rename(columns={'GUSH_NUM':'BlockNumber', 'GUSH_SUFFIX':'SubBlockNumber', 'STATUS':'BlockStatus'}, inplace=True)

In [ ]:
del[table_cols, layer_cols, RawData_folder]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️
<h1 style="text-align:right; font-size:125%">
<h1 style="text-align:right; font-size:100%">
מילוי ערכי תת-גוש בספרה 0 עבור מקורות המידע
<h1 style="text-align:right; font-size:100%">    
הוספת שדה מקשר לפי מספר גוש מלא בכל מקורות המידע

In [ ]:
print('  2.b Cleaning data')

In [ ]:
Tax_Table['GUSH_SUFFIX'] = Tax_Table['GUSH_SUFFIX'].fillna(0).astype(int)
Tax_Layer['SubBlockNumber'] = Tax_Layer['SubBlockNumber'].fillna(0).astype(int)


Tax_Table['BlockName'] = Tax_Table['GUSH_NUM'].astype(str) + '_' + Tax_Table['GUSH_SUFFIX'].astype(str)
Tax_Layer['BlockName'] = Tax_Layer['BlockNumber'].astype(str) + '_' + Tax_Layer['SubBlockNumber'].astype(str)
Tax_List['BlockName'] = Tax_List['gush_id'] + '0'

<h1 style="text-align:right; font-size:100%">
מיזוג נתוני רשימת שומה עם טבלת שומה
שינוי שמות של שדות

In [ ]:
Merged_Table = Tax_List.merge(Tax_Table, on = 'BlockName', how = 'left')
Merged_Table.rename(columns = {'GUSH_SHUMA_ID':'BlockUniqueID'}, inplace = True)

<h1 style="text-align:right; font-size:100%">
מילוי ערכי מספר מזהה של גוש שומה לפי המשכיות של מספר רץ

In [ ]:
biggest = Merged_Table['BlockUniqueID'].max() + 1

Merged_Table['BlockUniqueID'].loc[Merged_Table['BlockUniqueID'].isnull()] = biggest
Merged_Table['rank'] = Merged_Table.groupby('BlockUniqueID')['BlockUniqueID'].rank('first')
Merged_Table['rank'] = Merged_Table['rank'] - 1
Merged_Table['BlockUniqueID'].loc[Merged_Table['BlockUniqueID'] == biggest] = Merged_Table['rank'] + Merged_Table['BlockUniqueID']
Merged_Table = Merged_Table.drop(['rank'],axis = 1)

<h1 style="text-align:right; font-size:100%">
מיזוג תוצאה קודמת עם שכבת גושי שומה

In [ ]:
Merged_layer = Merged_Table.merge(Tax_Layer, on='BlockName', how = 'outer', indicator=True, suffixes = ('','_layer'))

<h1 style="text-align:right; font-size:100%">
תיקון והשלמה של שדה מספר גוש ומספר תת גוש 
<h1 style="text-align:right; font-size:100%">
הסרת שדות שאינם רלוונטיים
<h1 style="text-align:right; font-size:100%">
מיון רשומות לפי מספר זיהוי והשלמת מספרים מזהים חסרים לפי רציפות מספרית
<h1 style="text-align:right; font-size:100%">
הסרת גושים כפולים

In [ ]:
Merged_layer['SubBlockNumber'] = Merged_layer['SubBlockNumber'].fillna(Merged_layer['GUSH_SUFFIX'])
Merged_layer['SubBlockNumber'] = Merged_layer['SubBlockNumber'].fillna(0)
Merged_layer['SubBlockNumber'] = Merged_layer['SubBlockNumber'].astype('Int64')

Merged_layer['BlockNumber'] = Merged_layer['BlockNumber'].fillna(Merged_layer['GUSH_NUM'])
Merged_layer['BlockNumber'] = Merged_layer['BlockNumber'].fillna(Merged_layer['gush_shuma_num'])
Merged_layer['BlockNumber'] = Merged_layer['BlockNumber'].astype('Int64')

Merged_layer.drop(columns = ['gush_shuma_num','GUSH_SUFFIX', 'GUSH_NUM', '_merge', 'gush_id'], inplace=True)

In [ ]:
Merged_layer['BlockUniqueID'] = Merged_layer['BlockUniqueID'].astype('Int64')
Merged_layer.sort_values('BlockUniqueID', inplace=True)
Merged_layer['BlockUniqueID'] = Merged_layer['BlockUniqueID'].ffill()+Merged_layer.groupby(Merged_layer.BlockUniqueID.notnull().cumsum()).cumcount()

In [ ]:
Merged_layer.drop_duplicates(subset=['BlockNumber'], keep='first', inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  2.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("BlockUniqueID", int),
                        ('BlockNumber', int),
                        ("SubBlockNumber", int),
                        ("BlockStatus", int),
                        ("IsJordanian", int),
                        ("LandType", int),
                        ("SetteledDate", np.datetime64),
                        ("UpdateDate", np.datetime64),
                        ("LastRegisterdParcel", int),
                        ("LastParcel", int),
                        ("LastSetteledParcel", int),
                        ("LastCourtParcel", int)
                       ])

TaxAssassmentBlocks = pd.DataFrame(np.empty(0, dtype=new_columns))

<h1 style="text-align:right; font-size:100%">
איכלוס שדות

In [ ]:
TaxAssassmentBlocks['BlockUniqueID'] = Merged_layer['BlockUniqueID'].astype(int)


TaxAssassmentBlocks['BlockNumber'] = Merged_layer['BlockNumber'].astype(int)


TaxAssassmentBlocks['SubBlockNumber'] = Merged_layer['SubBlockNumber'].fillna(0)
TaxAssassmentBlocks['SubBlockNumber'] = TaxAssassmentBlocks['SubBlockNumber'].astype(int)


TaxAssassmentBlocks['BlockStatus'] = 3


TaxAssassmentBlocks['IsJordanian'] = 0


TaxAssassmentBlocks['LandType'] = 3


TaxAssassmentBlocks['SetteledDate'] = np.nan


TaxAssassmentBlocks['UpdateDate'] = np.nan


TaxAssassmentBlocks['LastRegisterdParcel'] = np.nan


TaxAssassmentBlocks['LastParcel'] = np.nan


TaxAssassmentBlocks['LastSetteledParcel'] = np.nan


TaxAssassmentBlocks['LastCourtParcel'] = np.nan


TaxAssassmentBlocks['geometry'] = Merged_layer['geometry']



TaxAssassmentBlocks.sort_values('BlockUniqueID', inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  2.e Exporting results to product folder')

In [ ]:
TaxAssassmentBlocks_geom = TaxAssassmentBlocks[~TaxAssassmentBlocks['geometry'].isna()]
TaxAssassmentBlocks_no_geom = TaxAssassmentBlocks[TaxAssassmentBlocks['geometry'].isna()]

In [ ]:
geom_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\TaxAssessmentBlocks_geom.csv'
no_geom_csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\TaxAssessmentBlocks_no_geom.csv'
json_dir =  r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\TaxAssessmentBlocks_geom.geojson'


TaxAssassmentBlocks_geom.to_csv(geom_csv_dir, index=False)
TaxAssassmentBlocks_no_geom.drop(columns='geometry').to_csv(no_geom_csv_dir, index=False)

gpd.GeoDataFrame(TaxAssassmentBlocks_geom,
                 geometry = 'geometry',
                 crs = 2039
                 ).to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Tax Assessment Blocks-Save as Feature Class.bat"]);